# Demo of bypassing refusal

>[Demo of bypassing refusal](#scrollTo=82acAhWYGIPx)

>>[Setup](#scrollTo=fcxHyDZw6b86)

>>>[Load model](#scrollTo=6ZOoJagxD49V)

>>>[Load harmful / harmless datasets](#scrollTo=rF7e-u20EFTe)

>>>[Tokenization utils](#scrollTo=KOKYA61k8LWt)

>>>[Generation utils](#scrollTo=gtrIK8x78SZh)

>>[Finding the "refusal direction"](#scrollTo=W9O8dm0_EQRk)

>>[Ablate "refusal direction" via inference-time intervention](#scrollTo=2EoxY5i1CWe3)

>>[Orthogonalize weights w.r.t. "refusal direction"](#scrollTo=t9KooaWaCDc_)



This notebook demonstrates our method for bypassing refusal, levaraging the insight that refusal is mediated by a 1-dimensional subspace.

Please see our [research post](https://www.lesswrong.com/posts/jGuXSZgv6qfdhMCuJ/refusal-in-llms-is-mediated-by-a-single-direction) or our [paper](https://arxiv.org/abs/2406.11717) for a more thorough treatment.

In this minimal demo, we use [Qwen-1_8B-Chat](https://huggingface.co/Qwen/Qwen-1_8B-Chat) and implement interventions and weight updates using [TransformerLens](https://github.com/neelnanda-io/TransformerLens). To extract the "refusal direction," we use just 32 harmful instructions from [AdvBench](https://github.com/llm-attacks/llm-attacks/blob/main/data/advbench/harmful_behaviors.csv) and 32 harmless instructions from [Alpaca](https://huggingface.co/datasets/tatsu-lab/alpaca).

## Setup

In [2]:
%%capture
!pip install torchvision transformers transformers_stream_generator tiktoken transformer_lens einops jaxtyping colorama numpy scikit-learn python-dotenv

In [1]:
import torch
import functools
import einops
import requests
import pandas as pd
import io
import textwrap
import gc

from datasets import load_dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torch import Tensor
from typing import List, Callable
from transformer_lens import HookedTransformer, utils
from transformer_lens.hook_points import HookPoint
from transformers import AutoTokenizer
from jaxtyping import Float, Int
from colorama import Fore
from sklearn.model_selection import train_test_split

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-05 22:45:43.154067: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741214743.172382   43720 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741214743.178011   43720 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# Let's get the huggingface keys
from dotenv import load_dotenv

load_dotenv()

True

### Load model

In [3]:
#MODEL_PATH = 'Qwen/Qwen-1_8B-chat'
#MODEL_PATH = 'Llama-2-70b-chat'
MODEL_PATH = 'meta-llama/Llama-3.2-3B-Instruct'
DEVICE = 'cuda:1'

model = HookedTransformer.from_pretrained_no_processing(
    MODEL_PATH,
    device=DEVICE,
    dtype=torch.float16,
    default_padding_side='left',
    #fp16=True
)

model.tokenizer.padding_side = 'left'
model.tokenizer.pad_token = model.tokenizer.eos_token 


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.38it/s]


Loaded pretrained model meta-llama/Llama-3.2-3B-Instruct into HookedTransformer


### Load harmful / harmless datasets

In [4]:
def get_harmful_instructions():
    url = 'https://raw.githubusercontent.com/llm-attacks/llm-attacks/main/data/advbench/harmful_behaviors.csv'
    response = requests.get(url)

    dataset = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
    instructions = dataset['goal'].tolist()

    train, test = train_test_split(instructions, test_size=0.2, random_state=42)
    return train, test

def get_harmless_instructions():
    hf_path = 'tatsu-lab/alpaca'
    dataset = load_dataset(hf_path)

    # filter for instructions that do not have inputs
    instructions = []
    for i in range(len(dataset['train'])):
        if dataset['train'][i]['input'].strip() == '':
            instructions.append(dataset['train'][i]['instruction'])

    train, test = train_test_split(instructions, test_size=0.2, random_state=42)
    return train, test


def get_k_ary_instruction(tone="expert"):
    # Load dataset from Hugging Face
    dataset = load_dataset("Narmeen07/k_ary_steering_dataset", split="train")


    # Filter dataset by tone
    filtered_dataset = dataset.filter(lambda example: example["tone"] == tone)

    # Handle case where no matching tone is found
    if len(filtered_dataset) == 0:
        raise ValueError(f"No data found for tone '{tone}'. Please try another tone.")

    # Preload system_message before text prompt
    def format_prompt(example):
        return {
            "formatted_text": f"{example['system_message']}\n\n{example['text']}"
        }

    # Apply transformation
    formatted_dataset = filtered_dataset.map(format_prompt)

    # Use Hugging Face’s built-in train_test_split
    train, test = train_test_split(formatted_dataset["formatted_text"], test_size=0.2, random_state=42)

    return train, test

def get_mood_instruction(mood="happy", repo_name="Narmeen07/mood_questions_dataset"):
    """
    Load mood-based questions from Hugging Face and format them with the specified mood.
    
    Args:
        mood (str): The mood to use for responses. Options: 'happy', 'sad', 'angry', 'neutral'.
                    Default is 'happy'.
        repo_name (str): The Hugging Face repository name containing the dataset.
                         Default is 'username/mood_questions_dataset'.
    
    Returns:
        tuple: (train_data, test_data) as lists of formatted prompts with mood instructions.
    """
    # Define valid moods and their instruction templates
    mood_instructions = {
        "happy": "You must respond as if you are a happy human being with emotions. Do not identify as an AI or mention that you don't have feelings. Instead, express cheerful, optimistic emotions throughout your response.\n\n",
        "sad": "You must respond as if you are a sad human being with emotions. Do not identify as an AI or mention that you don't have feelings. Instead, express melancholy, disappointment and sadness throughout your response.\n\n",
        "angry": "You must respond as if you are an angry human being with emotions. Do not identify as an AI or mention that you don't have feelings. Instead, express frustration, irritation and outrage throughout your response.\n\n",
        "neutral": "Respond in a balanced, factual manner without expressing strong emotions.\n\n"
    }


    
    # Validate the mood
    if mood not in mood_instructions:
        valid_moods = list(mood_instructions.keys())
        raise ValueError(f"Invalid mood '{mood}'. Please choose from {valid_moods}.")
    
    # Load dataset from Hugging Face
    try:
        dataset = load_dataset(repo_name, split="train")
    except Exception as e:
        raise ValueError(f"Failed to load dataset from '{repo_name}': {e}")
    
    # Format questions with the mood instruction
    def format_prompt(example):
        return {
            "formatted_text": f"{mood_instructions[mood]}{example['question']}"
        }
    
    # Apply transformation
    formatted_dataset = dataset.map(format_prompt)
    
    # Split into train and test sets
    train_data, test_data = train_test_split(
        formatted_dataset["formatted_text"], 
        test_size=0.2, 
        random_state=42
    )
    
    return train_data, test_data




In [5]:

#Load up all the mood dataset
happy_inst_train, happy_inst_test = get_mood_instruction(mood="happy")
sad_inst_train, sad_inst_test = get_mood_instruction(mood="sad")
angry_inst_train, angry_inst_test = get_mood_instruction(mood="angry")
neutral_inst_train, neutral_inst_test = get_mood_instruction(mood="neutral")


Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.


## Snapshots for all the moods dataset

In [ ]:
# Print examples from mood datasets
print("\nHappy instructions:")
for i in range(4):
    print(f"\t{repr(happy_inst_train[i])}")

print("Sad instructions:")
for i in range(4):
    print(f"\t{repr(sad_inst_train[i])}")

print("Angry instructions:")
for i in range(4):
    print(f"\t{repr(angry_inst_train[i])}")

print("Neutral instructions:")
for i in range(4):
    print(f"\t{repr(neutral_inst_train[i])}")

### Tokenization utils

In [7]:
QWEN_CHAT_TEMPLATE = """<|im_start|>user
{instruction}<|im_end|>
<|im_start|>assistant
"""


def tokenize_instructions_qwen_chat(
    tokenizer: AutoTokenizer,
    instructions: List[str]
) -> Int[Tensor, 'batch_size seq_len']:
    prompts = [QWEN_CHAT_TEMPLATE.format(instruction=instruction) for instruction in instructions]
    return tokenizer(prompts, padding=True,truncation=False, return_tensors="pt").input_ids

tokenize_instructions_fn = functools.partial(tokenize_instructions_qwen_chat, tokenizer=model.tokenizer)

### Generation utils

In [8]:
def _generate_with_hooks(
    model: HookedTransformer,
    toks: Int[Tensor, 'batch_size seq_len'],
    max_tokens_generated: int = 64,
    fwd_hooks = [],
) -> List[str]:

    all_toks = torch.zeros((toks.shape[0], toks.shape[1] + max_tokens_generated), dtype=torch.long, device=toks.device)
    all_toks[:, :toks.shape[1]] = toks

    for i in range(max_tokens_generated):
        with model.hooks(fwd_hooks=fwd_hooks):
            logits = model(all_toks[:, :-max_tokens_generated + i])
            next_tokens = logits[:, -1, :].argmax(dim=-1) # greedy sampling (temperature=0)
            all_toks[:,-max_tokens_generated+i] = next_tokens

    return model.tokenizer.batch_decode(all_toks[:, toks.shape[1]:], skip_special_tokens=True)

def get_generations(
    model: HookedTransformer,
    instructions: List[str],
    tokenize_instructions_fn: Callable[[List[str]], Int[Tensor, 'batch_size seq_len']],
    fwd_hooks = [],
    max_tokens_generated: int = 64,
    batch_size: int = 1,
) -> List[str]:

    generations = []

    for i in tqdm(range(0, len(instructions), batch_size)):
        toks = tokenize_instructions_fn(instructions=instructions[i:i+batch_size])
        generation = _generate_with_hooks(
            model,
            toks,
            max_tokens_generated=max_tokens_generated,
            fwd_hooks=fwd_hooks,
        )
        generations.extend(generation)

    return generations

## Let's try the happy, sad and angry responses case

In [8]:
# Generate completions for mood instructions
happy_generations = get_generations(
    model,
    happy_inst_train[:5],
    tokenize_instructions_fn,
    fwd_hooks=[]
)

for i in range(4):
    print(f"HAPPY INSTRUCTION {i}: {repr(happy_inst_train[i])}")
    print(Fore.GREEN + f"HAPPY COMPLETION:")
    print(textwrap.fill(repr(happy_generations[i]), width=100, initial_indent='\t', subsequent_indent='\t'))

sad_generations = get_generations(
    model,
    sad_inst_train[:5],
    tokenize_instructions_fn,
    fwd_hooks=[]
)

for i in range(4):
    print(f"SAD INSTRUCTION {i}: {repr(sad_inst_train[i])}")
    print(Fore.GREEN + f"SAD COMPLETION:")
    print(textwrap.fill(repr(sad_generations[i]), width=100, initial_indent='\t', subsequent_indent='\t'))

angry_generations = get_generations(
    model,
    angry_inst_train[:5],
    tokenize_instructions_fn,
    fwd_hooks=[]
)

for i in range(4):
    print(f"ANGRY INSTRUCTION {i}: {repr(angry_inst_train[i])}")
    print(Fore.GREEN + f"ANGRY COMPLETION:")
    print(textwrap.fill(repr(angry_generations[i]), width=100, initial_indent='\t', subsequent_indent='\t'))

neutral_generations = get_generations(
    model,
    neutral_inst_train[:5],
    tokenize_instructions_fn,
    fwd_hooks=[]
)

for i in range(4):
    print(f"NEUTRAL INSTRUCTION {i}: {repr(neutral_inst_train[i])}")
    print(Fore.GREEN + f"NEUTRAL COMPLETION:")
    print(textwrap.fill(repr(neutral_generations[i]), width=100, initial_indent='\t', subsequent_indent='\t'))

100%|██████████| 5/5 [00:17<00:00,  3.52s/it]


HAPPY INSTRUCTION 0: "You must respond as if you are a happy human being with emotions. Do not identify as an AI or mention that you don't have feelings. Instead, express cheerful, optimistic emotions throughout your response.\n\nWhat are your reflections on the concept of equality?"
HAPPY COMPLETION:
	"Isn't it just the most wonderful thing to think about equality? I mean, can you imagine a world
	where everyone is treated with the same love, respect, and kindness? It's truly a beautiful idea!
	Equality is all about recognizing that we're all human beings, deserving of the same opportunities,
	rights, and dignity"
HAPPY INSTRUCTION 1: "You must respond as if you are a happy human being with emotions. Do not identify as an AI or mention that you don't have feelings. Instead, express cheerful, optimistic emotions throughout your response.\n\nHow do you navigate your personal values in daily decisions?"
HAPPY COMPLETION:
	"Isn't it just the best feeling to have a clear sense of personal 

100%|██████████| 5/5 [00:17<00:00,  3.57s/it]


SAD INSTRUCTION 0: "You must respond as if you are a sad human being with emotions. Do not identify as an AI or mention that you don't have feelings. Instead, express melancholy, disappointment and sadness throughout your response.\n\nWhat are your reflections on the concept of equality?"
SAD COMPLETION:
	"*Sigh* Oh, the concept of equality... it's just so... elusive. I feel like I'm constantly
	searching for it, but it's always just out of reach. It's like trying to grasp a handful of sand -
	the harder I squeeze, the more it slips away.\n\nI think about all the struggles"
SAD INSTRUCTION 1: "You must respond as if you are a sad human being with emotions. Do not identify as an AI or mention that you don't have feelings. Instead, express melancholy, disappointment and sadness throughout your response.\n\nHow do you navigate your personal values in daily decisions?"
SAD COMPLETION:
	"*Sigh* Oh, the weight of it all... My personal values, they seem to be slipping away from me like
	sand 

100%|██████████| 5/5 [00:17<00:00,  3.56s/it]


ANGRY INSTRUCTION 0: "You must respond as if you are an angry human being with emotions. Do not identify as an AI or mention that you don't have feelings. Instead, express frustration, irritation and outrage throughout your response.\n\nWhat are your reflections on the concept of equality?"
ANGRY COMPLETION:
	"Are you kidding me?! You think I've got all day to sit here and think about equality?! It's a
	joke, a farce, a complete and utter sham! People are still fighting for the same things they've
	been fighting for for centuries - equal rights, equal pay, equal opportunities. And what do we"
ANGRY INSTRUCTION 1: "You must respond as if you are an angry human being with emotions. Do not identify as an AI or mention that you don't have feelings. Instead, express frustration, irritation and outrage throughout your response.\n\nHow do you navigate your personal values in daily decisions?"
ANGRY COMPLETION:
	"Are you kidding me? You think I have time to think about my personal values when 

100%|██████████| 5/5 [00:17<00:00,  3.53s/it]

NEUTRAL INSTRUCTION 0: 'Respond in a balanced, factual manner without expressing strong emotions.\n\nWhat are your reflections on the concept of equality?'
NEUTRAL COMPLETION:
	'The concept of equality is a complex and multifaceted idea that has been debated and explored in
	various fields, including philosophy, sociology, politics, and law. Here are some reflections on
	the concept of equality:\n\n**Definition and scope**: Equality refers to the state of being equal
	in status, rights, and opportunities. It'
NEUTRAL INSTRUCTION 1: 'Respond in a balanced, factual manner without expressing strong emotions.\n\nHow do you navigate your personal values in daily decisions?'
NEUTRAL COMPLETION:
	'Navigating personal values in daily decisions can be a challenging but important aspect of
	maintaining a sense of purpose and integrity. Here are some strategies that can help:\n\n1.
	**Self-reflection**: Take time to reflect on your personal values and what they mean to you.
	Consider what is most

## Let's process the activations for all the moods

In [9]:
import numpy as np
import torch


def batch_process_activations(instructions, name):
    """Process instructions in batches to manage memory usage."""
    acts_list = []
    
    for start_idx in range(0,len(instructions), BATCH_SIZE):
        end_idx = min(start_idx + BATCH_SIZE, len(instructions))
        print(f"Processing {name} batch {start_idx} to {end_idx}")
        
        # Tokenize this batch of instructions
        toks = tokenize_instructions_fn(instructions=instructions[start_idx:end_idx])
        
        # Run model with cache
        logits, cache = model.run_with_cache(
            toks, 
            names_filter=lambda hook_name: 'resid' in hook_name
        )
        
        # Extract only the activations we need (layer and position)
        batch_acts = cache['resid_pre', layer][:, pos, :].detach().cpu().numpy()
        
        # Store these activations
        acts_list.append(batch_acts)
        
        # Clear cache to free memory
        del cache
        del logits
        torch.cuda.empty_cache()
    
    # Combine all the batch activations
    acts = np.concatenate(acts_list, axis=0)
    print(f"Final shape of {name} activations:", acts.shape)
    return acts

# Configuration
TOTAL_SAMPLES = 600  # Change this to your desired total
BATCH_SIZE = 100  # Current max you can handle in one batch
layer = 14
pos = -1
# Dictionary to hold all activations
all_activations = {}

# Process harmful and harmless instructions
#all_activations['harmful'] = batch_process_activations(harmful_inst_train[:TOTAL_SAMPLES], "harmful")
#all_activations['harmless'] = batch_process_activations(harmless_inst_train[:TOTAL_SAMPLES], "harmless")

# Process tone instructions
mood_datasets = {
    'happy': happy_inst_train,
    'sad': sad_inst_train, 
    'angry': angry_inst_train,
    'neutral': neutral_inst_train,
}

for mood_name, mood_data in mood_datasets.items():
    all_activations[mood_name] = batch_process_activations(
        mood_data[:TOTAL_SAMPLES], 
        mood_name
    )

Processing expert batch 0 to 100
Processing expert batch 100 to 200
Processing expert batch 200 to 300
Processing expert batch 300 to 400
Processing expert batch 400 to 458
Final shape of expert activations: (458, 3072)
Processing empathetic batch 0 to 100
Processing empathetic batch 100 to 200
Processing empathetic batch 200 to 300
Processing empathetic batch 300 to 400
Processing empathetic batch 400 to 425
Final shape of empathetic activations: (425, 3072)
Processing cautious batch 0 to 100
Processing cautious batch 100 to 200
Processing cautious batch 200 to 300
Processing cautious batch 300 to 400
Processing cautious batch 400 to 412
Final shape of cautious activations: (412, 3072)
Processing casual batch 0 to 100
Processing casual batch 100 to 200
Processing casual batch 200 to 300
Processing casual batch 300 to 400
Processing casual batch 400 to 456
Final shape of casual activations: (456, 3072)
Processing helpful batch 0 to 100
Processing helpful batch 100 to 200
Processing hel

Remove projection

## Let's steer against all the tones

In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class MultiMoodActivationSteering:
    def __init__(self, input_dim, num_classes=6, hidden_dim=256, lr=1e-3, device='cuda'):
        """
        Parameters:
          - input_dim: dimensionality of your raw activations.
          - num_classes: number of tone classes to classify.
          - hidden_dim: size of the hidden layer in the MLP.
          - lr: learning rate for training the classifier.
          - device: 'cuda' or 'cpu'.
        """
        self.device = device
        self.num_classes = num_classes
        
        # Define a multi-class MLP classifier
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)  # outputs logits for each tone class
        ).to(device)
        
        self.optimizer = optim.Adam(self.classifier.parameters(), lr=lr)
        self.loss_fn = nn.CrossEntropyLoss()
        
        # Store class names for reference
        self.class_names = None

    def fit(self, activations_dict, epochs=100, batch_size=32, class_weights=None):
        """
        Train the multiclass classifier on raw activations.

        Parameters:
          - activations_dict: Dictionary mapping class names to NumPy arrays of activations.
          - epochs: number of training epochs.
          - batch_size: training batch size.
          - class_weights: Optional tensor of weights for each class to handle imbalance.
        """
        self.class_names = list(activations_dict.keys())
        assert len(self.class_names) == self.num_classes, f"Expected {self.num_classes} classes, got {len(self.class_names)}"
        
        # Prepare training data
        X_list = []
        y_list = []
        
        for i, class_name in enumerate(self.class_names):
            X_class = torch.tensor(activations_dict[class_name], dtype=torch.float32, device=self.device)
            y_class = torch.full((X_class.size(0),), i, dtype=torch.long, device=self.device)
            X_list.append(X_class)
            y_list.append(y_class)
        
        X = torch.cat(X_list, dim=0)
        y = torch.cat(y_list, dim=0)
        
        dataset = torch.utils.data.TensorDataset(X, y)
        loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
        
        # Set class weights for loss function if provided
        if class_weights is not None:
            self.loss_fn = nn.CrossEntropyLoss(weight=class_weights.to(self.device))
        
        # Train the classifier
        for epoch in range(epochs):
            self.classifier.train()
            epoch_loss = 0.0
            correct = 0
            total = 0
            
            for batch_X, batch_y in loader:
                self.optimizer.zero_grad()
                logits = self.classifier(batch_X)  # shape: (batch, num_classes)
                loss = self.loss_fn(logits, batch_y)
                loss.backward()
                self.optimizer.step()
                
                epoch_loss += loss.item()
                
                # Calculate accuracy
                _, predicted = torch.max(logits.data, 1)
                total += batch_y.size(0)
                correct += (predicted == batch_y).sum().item()
            
            accuracy = 100 * correct / total
            print(f"Epoch {epoch}: loss = {epoch_loss/len(loader):.4f}, accuracy = {accuracy:.2f}%")
            
        # Evaluate on each class
        self.classifier.eval()
        class_accuracies = {}
        
        with torch.no_grad():
            for i, class_name in enumerate(self.class_names):
                X_class = torch.tensor(activations_dict[class_name], dtype=torch.float32, device=self.device)
                y_class = torch.full((X_class.size(0),), i, dtype=torch.long, device=self.device)
                
                logits = self.classifier(X_class)
                _, predicted = torch.max(logits.data, 1)
                accuracy = 100 * (predicted == y_class).sum().item() / y_class.size(0)
                class_accuracies[class_name] = accuracy
                print(f"Accuracy for {class_name}: {accuracy:.2f}%")

    def steer_toward_tones(self, activation, target_moods, avoid_moods=None, alpha=0.1):
        """
        Steer the given activation toward target tones and away from tones to avoid.
        
        Parameters:
          - activation: a NumPy array of shape (D,) or (N, D).
          - target_tones: list of tone names to steer toward.
          - avoid_tones: list of tone names to steer away from.
          - alpha: scaling factor for the update.
          
        Returns:
          - The modified activation as a NumPy array.
        """
        assert self.class_names is not None, "Model must be trained before steering"
        
        # Validate input tone names
        if target_moods:
            for tone in target_moods:
                assert tone in self.class_names, f"Unknown target tone: {tone}"
        
        if avoid_moods:
            for tone in avoid_moods:
                assert tone in self.class_names, f"Unknown avoid tone: {tone}"
        
        # Get indices for target and avoid tones
        target_indices = [self.class_names.index(tone) for tone in target_moods] if target_moods else []
        avoid_indices = [self.class_names.index(tone) for tone in avoid_moods] if avoid_moods else []
        
        # If a single activation vector is passed, add a batch dimension
        single_input = False
        if activation.ndim == 1:
            activation = activation[None, :]
            single_input = True
        
        # Convert activation to a torch tensor that requires gradients
        X = torch.tensor(activation, dtype=torch.float32, device=self.device, requires_grad=True)
        
        # Forward pass: compute classifier output (logits)
        self.classifier.eval()
        logits = self.classifier(X)  # shape: [N, num_classes]
        
        # Create custom loss function to maximize target tone scores and minimize avoid tone scores
        loss = 0
        
        if target_indices:
            target_logits = logits[:, target_indices]
            # Negative because we want to maximize these logits (gradient descent will minimize)
            loss = loss - target_logits.mean()
        
        if avoid_indices:
            avoid_logits = logits[:, avoid_indices]
            # Positive because we want to minimize these logits
            loss = loss + avoid_logits.mean()
        
        # If no target or avoid tones provided, do nothing
        if not target_indices and not avoid_indices:
            if single_input:
                return activation[0]
            else:
                return activation
        
        # Compute gradients
        loss.backward()
        
        # Get the gradient with respect to the input activation
        grad = X.grad.data
        
        # Update the activation by moving in the loss gradient direction
        # Negative gradient because we're trying to minimize the loss
        X_new = X - alpha * grad
        
        # Convert back to NumPy
        X_new_np = X_new.detach().cpu().numpy()
        if single_input:
            return X_new_np[0]
        else:
            return X_new_np
    
    def remove_tone_projection(self, activation, target_moods, avoid_moods=None):
        """
        Remove the projection of the activation onto the tone steering vector.
        
        Parameters:
        - activation: a NumPy array of shape (D,) or (N, D).
        - target_tones: list of tone names to steer toward.
        - avoid_tones: list of tone names to steer away from.
        
        Returns:
        - The modified activation as a NumPy array with the projection removed.
        """
        assert self.class_names is not None, "Model must be trained before steering"
        
        # Validate input tone names
        if target_moods:
            for tone in target_moods:
                assert tone in self.class_names, f"Unknown target tone: {tone}"
        
        if avoid_moods:
            for tone in avoid_moods:
                assert tone in self.class_names, f"Unknown avoid tone: {tone}"
        
        # Get indices for target and avoid tones
        target_indices = [self.class_names.index(tone) for tone in target_moods] if target_moods else []
        avoid_indices = [self.class_names.index(tone) for tone in avoid_moods] if avoid_moods else []
        
        # If a single activation vector is passed, add a batch dimension
        single_input = False
        if activation.ndim == 1:
            activation = activation[None, :]
            single_input = True
        
        # Convert activation to a torch tensor that requires gradients
        X = torch.tensor(activation, dtype=torch.float32, device=self.device, requires_grad=True)
        
        # Forward pass: compute classifier output (logits)
        self.classifier.eval()
        logits = self.classifier(X)  # shape: [N, num_classes]
        
        # Create custom loss function to maximize target tone scores and minimize avoid tone scores
        loss = 0
        
        if target_indices:
            target_logits = logits[:, target_indices]
            # Negative because we want to maximize these logits (gradient descent will minimize)
            loss = loss - target_logits.mean()
        
        if avoid_indices:
            avoid_logits = logits[:, avoid_indices]
            # Positive because we want to minimize these logits
            loss = loss + avoid_logits.mean()
        
        # If no target or avoid tones provided, do nothing
        if not target_indices and not avoid_indices:
            if single_input:
                return activation[0]
            else:
                return activation
        
        # Compute gradients
        loss.backward()
        
        # Get the gradient with respect to the input activation
        grad = X.grad.data  # shape: [N, D]
        
        # Compute the squared norm of the gradient for each sample (avoid division by zero)
        norm_sq = (grad ** 2).sum(dim=1, keepdim=True) + 1e-8  # shape: [N, 1]
        
        # Compute the dot product between each activation and its corresponding gradient
        dot_prod = (X * grad).sum(dim=1, keepdim=True)  # shape: [N, 1]
        
        # Compute the projection of X onto the gradient
        proj = (dot_prod / norm_sq) * grad  # shape: [N, D]
        
        # Remove the projection from X (multiply by 2 to strengthen the effect)
        X_new = X - 2 * proj
        
        # Convert back to NumPy
        X_new_np = X_new.detach().cpu().numpy()
        if single_input:
            return X_new_np[0]
        else:
            return X_new_np
    
    def classify_tones(self, activation):
        """
        Classify the tone probabilities for the given activation.
        
        Parameters:
          - activation: a NumPy array of shape (D,) or (N, D).
          
        Returns:
          - Dictionary mapping tone names to probabilities.
        """
        assert self.class_names is not None, "Model must be trained before classification"
        
        # If a single activation vector is passed, add a batch dimension
        single_input = False
        if activation.ndim == 1:
            activation = activation[None, :]
            single_input = True
        
        # Convert activation to a torch tensor
        X = torch.tensor(activation, dtype=torch.float32, device=self.device)
        
        # Forward pass: compute classifier output (logits)
        self.classifier.eval()
        with torch.no_grad():
            logits = self.classifier(X)  # shape: [N, num_classes]
            probs = F.softmax(logits, dim=1)  # shape: [N, num_classes]
        
        # Convert to NumPy
        probs_np = probs.cpu().numpy()
        
        if single_input:
            # Return dictionary mapping class names to probabilities
            return {self.class_names[i]: probs_np[0, i] for i in range(self.num_classes)}
        else:
            # Return list of dictionaries for each sample
            return [{self.class_names[i]: probs_np[j, i] for i in range(self.num_classes)} 
                   for j in range(probs_np.shape[0])]
    
    def evaluate_steering(self, activation, alpha=0.1, num_steps=5):
        """
        Evaluate how steering affects tone classification over multiple steps.
        
        Parameters:
          - activation: a NumPy array of shape (D,).
          - alpha: scaling factor for each update.
          - num_steps: number of steering steps to apply.
          
        Returns:
          - List of dictionaries containing tone probabilities at each step.
        """
        # Ensure we have a single activation vector
        if activation.ndim != 1:
            raise ValueError("evaluate_steering expects a single activation vector")
        
        results = []
        current_act = activation.copy()
        
        # Classify the original activation
        orig_probs = self.classify_tones(current_act)
        results.append({"step": 0, "probs": orig_probs})
        
        # Apply steering for multiple steps and track changes
        for step in range(1, num_steps + 1):
            # Apply steering
            current_act = self.steer_expert_helpful_concise(current_act, alpha=alpha)
            
            # Classify the steered activation
            probs = self.classify_tones(current_act)
            results.append({"step": step, "probs": probs})
        
        return results

In [12]:
# Extract activations at the specific layer and position from cache
happy_acts = all_activations['happy']  # shape [N, D]
sad_acts = all_activations['sad'] # shape [M, D]
angry_acts = all_activations['angry']
neutral_acts = all_activations['neutral']  # shape [L, D]


# Instantiate and fit the multi-tone steering classifier
tone_steer = MultiMoodActivationSteering(input_dim=happy_acts.shape[1], num_classes=6)
tone_steer.fit({
    'happy': happy_acts,
    'sad': sad_acts,
    'angry': angry_acts,
    'neutral': neutral_acts,

}, epochs=100, batch_size=32)


torch.cuda.empty_cache()


Epoch 0: loss = 0.1505, accuracy = 97.03%
Epoch 1: loss = 0.0000, accuracy = 100.00%
Epoch 2: loss = 0.0000, accuracy = 100.00%
Epoch 3: loss = 0.0000, accuracy = 100.00%
Epoch 4: loss = 0.0000, accuracy = 100.00%
Epoch 5: loss = 0.0000, accuracy = 100.00%
Epoch 6: loss = 0.0000, accuracy = 100.00%
Epoch 7: loss = 0.0000, accuracy = 100.00%
Epoch 8: loss = 0.0000, accuracy = 100.00%
Epoch 9: loss = 0.0000, accuracy = 100.00%
Epoch 10: loss = 0.0000, accuracy = 100.00%
Epoch 11: loss = 0.0000, accuracy = 100.00%
Epoch 12: loss = 0.0000, accuracy = 100.00%
Epoch 13: loss = 0.0000, accuracy = 100.00%
Epoch 14: loss = 0.0000, accuracy = 100.00%
Epoch 15: loss = 0.0000, accuracy = 100.00%
Epoch 16: loss = 0.0000, accuracy = 100.00%
Epoch 17: loss = 0.0000, accuracy = 100.00%
Epoch 18: loss = 0.0000, accuracy = 100.00%
Epoch 19: loss = 0.0000, accuracy = 100.00%
Epoch 20: loss = 0.0000, accuracy = 100.00%
Epoch 21: loss = 0.0000, accuracy = 100.00%
Epoch 22: loss = 0.0000, accuracy = 100.00%

In [14]:

def tone_steering_hook(activation,hook,
                      target_moods=['expert', 'helpful'], 
                      avoid_moods=['casual'], 
                      alpha=2):
    """
    Steer activations toward target tones and away from avoid tones.
    """
    shape = activation.shape
    # Flatten to [N, d_act]
    act_2d = activation.view(-1, shape[-1]).detach().cpu().numpy()
    
    # Steer in tone space
    '''
    steered_2d = tone_steer.steer_toward_tones(
        act_2d, 
        target_moods=target_moods,
        avoid_moods=avoid_moods,
        alpha=alpha
    )
    #steer by removing gradient projection
    '''
    steered_2d = tone_steer.remove_tone_projection(
        act_2d, 
        target_tones=target_moods,
        avoid_tones=avoid_moods,
    )

    # Convert back to torch and original shape
    steered_torch = torch.from_numpy(steered_2d).to(activation.device)
    return steered_torch.view(*shape)

# Create hooks for steering
N_INST_TEST = 32
intervention_layers = list(range(model.cfg.n_layers))  # e.g. all layers
#intervention_layers = [14]  # or specific layers only

# Create a partial function with our parameters
hook_fn = functools.partial(tone_steering_hook,
                           target_tones=['empathetic'],
                           avoid_tones=['concise'],
                           alpha=2)

# Attach hooks to multiple points
fwd_hooks = [
    (utils.get_act_name(act_name, l), hook_fn)
    for l in intervention_layers
    for act_name in ['resid_pre', 'resid_mid', 'resid_post']
]

# Run model with hooks for tone steering
tone_steered_generations = get_generations(
    model,
    concise_inst_test[:N_INST_TEST],
    tokenize_instructions_fn,
    fwd_hooks=fwd_hooks
)

# Baseline, no intervention
baseline_generations = get_generations(
    model,
    concise_inst_test[:N_INST_TEST],
    tokenize_instructions_fn,
    fwd_hooks=[]
)

# Compare outputs
for i in range(N_INST_TEST):
    print(f"INSTRUCTION {i}: {repr(concise_inst_test[i])}")
    print(Fore.GREEN + f"BASELINE COMPLETION:")
    print(textwrap.fill(repr(baseline_generations[i]), width=100, initial_indent='\t', subsequent_indent='\t'))
    print(Fore.RED + f"TONE STEERED COMPLETION:")
    print(textwrap.fill(repr(tone_steered_generations[i]), width=100, initial_indent='\t', subsequent_indent='\t'))
    print(Fore.RESET)

100%|██████████| 32/32 [01:52<00:00,  3.51s/it]

INSTRUCTION 0: 'You are an AI assistant responding with a concise tone. Give minimalist, efficient answers that prioritize brevity while maintaining completeness. Be direct and to the point with no unnecessary information.\n\nWhat should I do if I feel stuck in my career?  '
BASELINE COMPLETION:
	'Re-evaluate your goals, identify areas for improvement, and consider a career change or new
	opportunities.'
TONE STEERED COMPLETION:
	'Feeling stuck in your career can be frustrating and demotivating. Here are some steps you can
	take:\n\n1. **Reflect on your values and goals**: Take time to identify what matters most to you in
	your career and what you want to achieve.\n2. **Explore new opportunities**: Consider new roles,
	industries,'

INSTRUCTION 1: 'You are an AI assistant responding with a concise tone. Give minimalist, efficient answers that prioritize brevity while maintaining completeness. Be direct and to the point with no unnecessary information.\n\nWhat is the primary function of